In [115]:
import yfinance as yf

import pandas as pd
import numpy as np
import os

from skfolio import Population, RiskMeasure, Portfolio
from skfolio.preprocessing import prices_to_returns
from skfolio.optimization import (
    MeanRisk,
    ObjectiveFunction,
    EqualWeighted,
    InverseVolatility,
    RiskBudgeting
)

import dotenv 
dotenv.load_dotenv()

True

In [116]:
DATA_PATH = os.getenv('DATA_PATH')

def get_path(filename):
    return os.path.join(DATA_PATH, 'financials',filename)

get_path('backtest.csv')

'./res/data/financials/backtest.csv'

In [117]:
df = pd.read_csv(get_path('backtest1.csv'))
df

,date,symbol,marketcapname,sector
0,01-01-2015,GLAXO,Largecap,Pharmaceuticals
1,01-01-2016,GLAXO,Largecap,Pharmaceuticals
2,01-01-2016,VINATIORGA,Midcap,Industrials
3,01-01-2016,AJANTPHARM,Largecap,Pharmaceuticals
4,01-01-2016,SUNTV,Largecap,Media
5,01-01-2016,ZFCVINDIA,Largecap,Auto
6,02-01-2017,KANSAINER,Largecap,Industrials
7,02-01-2017,AJANTPHARM,Largecap,Pharmaceuticals
8,02-01-2017,AIAENG,Largecap,Industrials
9,02-01-2017,SUNTV,Largecap,Media


In [118]:
df['year'] = df['date'].str.split('-', expand=True)[[2]].astype(int)
df = df.drop(columns=['date'])
df

,symbol,marketcapname,sector,year
0,GLAXO,Largecap,Pharmaceuticals,2015
1,GLAXO,Largecap,Pharmaceuticals,2016
2,VINATIORGA,Midcap,Industrials,2016
3,AJANTPHARM,Largecap,Pharmaceuticals,2016
4,SUNTV,Largecap,Media,2016
5,ZFCVINDIA,Largecap,Auto,2016
6,KANSAINER,Largecap,Industrials,2017
7,AJANTPHARM,Largecap,Pharmaceuticals,2017
8,AIAENG,Largecap,Industrials,2017
9,SUNTV,Largecap,Media,2017


In [119]:
def get_returns(year):
    tickers = [ t + '.NS' for t in df.loc[df.year == year].symbol.to_list()]
    start_date = f'{year}-01-01'
    end_date = f'{year+1}-01-01'
    return yf.Tickers(tickers).history(start=start_date,end=end_date)

def get_portfolio(close):
    X = prices_to_returns(close)
    model = RiskBudgeting(risk_measure=RiskMeasure.CVAR)
    return model.fit_predict(X)

In [120]:
returns_cache = {}

In [121]:
d = []

for year in range(2019, 2025):
    if year not in returns_cache:
        returns_cache[year] = get_returns(year)['Close']
    portfolio = get_portfolio(returns_cache[year])
    d.append(portfolio.returns_df)

ret = pd.DataFrame(pd.concat(d))
ret

[*********************100%%**********************]  0 of 0 completed


ValueError: No objects to concatenate

In [ ]:
from skfolio import BasePortfolio


BasePortfolio(returns=ret, observations=ret.index, compounded=True).plot_cumulative_returns()